In [1]:
import torch

In [2]:
from transformers import AutoTokenizer, OPTForCausalLM, BloomForCausalLM, CodeGenForCausalLM

/home/ubuntu/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import transformers
transformers.__version__

'4.33.1'

In [4]:
# model = OPTForCausalLM.from_pretrained("facebook/opt-125m")
# tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")

In [5]:
# tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
# model = BloomForCausalLM.from_pretrained("bigscience/bloom-560m")

In [6]:
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")
model = CodeGenForCausalLM.from_pretrained("Salesforce/codegen-350M-mono")

In [7]:
model

CodeGenForCausalLM(
  (transformer): CodeGenModel(
    (wte): Embedding(51200, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-19): 20 x CodeGenBlock(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): CodeGenAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (qkv_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=False)
        )
        (mlp): CodeGenMLP(
          (fc_in): Linear(in_features=1024, out_features=4096, bias=True)
          (fc_out): Linear(in_features=4096, out_features=1024, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=51200, bias=True)
)

In [8]:
def model_pre_hook(module, args, kwargs):
    # print(args, kwargs)
    display = {}
    for k, v in kwargs.items():
        if isinstance(v, torch.Tensor):
            display[k] = v.shape 
        elif k == 'past_key_values' and v is not None:
            display[k] = f'{len(v)} * ' + str((v[0][0].shape, v[0][1].shape))
        else:
            display[k] = v
    print(display)

    return args, kwargs

model._forward_pre_hooks.clear()
model.register_forward_pre_hook(model_pre_hook, with_kwargs=True)
model._forward_pre_hooks

# def layer_pre_hook(module, args, kwargs):
#     display = {}
#     for k, v in kwargs.items():
#         if isinstance(v, torch.Tensor):
#             display[k] = v.shape 
#         elif k in ['past_key_value', 'layer_past'] and v is not None:
#             display[k] = str((v[0].shape, v[1].shape))
#         else:
#             display[k] = v
#     print(display)

#     return args, kwargs

# layer = model.model.decoder.layers[2]

# layer._forward_pre_hooks.clear()
# layer.register_forward_pre_hook(layer_pre_hook, with_kwargs=True)
# layer._forward_pre_hooks


OrderedDict([(0, <function __main__.model_pre_hook(module, args, kwargs)>)])

In [12]:
prompts = [
    'Who are you? Are you conscious?',
    'Where is Deutschland?',
    'How is Huawei Mate 60 Pro?'
] * 4

prompt_len = 20

inputs = tokenizer(prompts, padding="max_length", max_length=prompt_len, return_tensors="pt")


# Generate
generate_ids = model.generate(
    inputs.input_ids, 
    pad_token_id=tokenizer.eos_token_id,
    max_length=30 + prompt_len,
    # num_beams=2,
    # num_beam_groups=2,
    # diversity_penalty=0.1,
    # do_sample=True,
)

output_texts = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
for output_text in output_texts:
    print(output_text)
    print('-' * 10)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


{'input_ids': torch.Size([12, 20]), 'past_key_values': None, 'use_cache': True, 'position_ids': torch.Size([12, 20]), 'attention_mask': torch.Size([12, 20]), 'token_type_ids': None, 'return_dict': True, 'output_attentions': False, 'output_hidden_states': False}
{'input_ids': torch.Size([12, 1]), 'past_key_values': '20 * (torch.Size([12, 16, 20, 64]), torch.Size([12, 16, 20, 64]))', 'use_cache': True, 'position_ids': torch.Size([12, 1]), 'attention_mask': torch.Size([12, 21]), 'token_type_ids': None, 'return_dict': True, 'output_attentions': False, 'output_hidden_states': False}
{'input_ids': torch.Size([12, 1]), 'past_key_values': '20 * (torch.Size([12, 16, 21, 64]), torch.Size([12, 16, 21, 64]))', 'use_cache': True, 'position_ids': torch.Size([12, 1]), 'attention_mask': torch.Size([12, 22]), 'token_type_ids': None, 'return_dict': True, 'output_attentions': False, 'output_hidden_states': False}
{'input_ids': torch.Size([12, 1]), 'past_key_values': '20 * (torch.Size([12, 16, 22, 64]), t